## ●Notebookの内容

前処理&形状変換後データ(コード5の「変数選択後の前処理&形状変換」で作成したもの)の読み込みと左右の入れ込みデータの時点ずらし、モデルの学習とバリデーション

# 1. 前準備

## 1.1 パッケージのインポート・乱数固定

In [1]:
import pandas as pd #pandasパッケージをインポート
import numpy as np #numpyパッケージをインポート
import torch #ライブラリ「PyTorch」のtorchパッケージをインポート
import torch.nn as nn #「ニューラルネットワーク」モジュールの別名定義
import torch.nn.functional as F #「ニューラルネットワーク・活性化関数」モジュールの別名定義
import collections
import os
import pickle
import optuna
import torch.optim as optim

#乱数の固定
torch.manual_seed(123)

## 1.2 MPSの使用指定

In [2]:
if torch.backends.mps.is_available():
    device = torch.device("mps")
print('MPSの使用は',torch.backends.mps.is_available(),'である(Trueなら使用可能、Falseなら使用不可)。')

MPSの使用は True である(Trueなら使用可能、Falseなら使用不可)。


# 2. Early Stoppingクラスの生成

In [3]:
class EarlyStopping:
    """Early stops the training if validation loss doesn't improve after a given patience."""
    def __init__(self, patience=7, verbose=False, delta=0, path='checkpoint.pt', trace_func=print):
        """
        Args:
            patience (int): How long to wait after last time validation loss improved.
                            Default: 7
            verbose (bool): If True, prints a message for each validation loss improvement. 
                            Default: False
            delta (float): Minimum change in the monitored quantity to qualify as an improvement.
                            Default: 0
            path (str): Path for the checkpoint to be saved to.
                            Default: 'checkpoint.pt'
            trace_func (function): trace print function.
                            Default: print            
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        #self.path = path
        self.trace_func = trace_func
        
    def __call__(self, val_loss, model):
        score = -val_loss
        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score:
            self.counter += 1
            print(f"EarlyStopping counter: {self.counter} out of {self.patience}")
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0
        
    def save_checkpoint(self, val_loss, model):
        """Saves model when validation loss decreases or accuracy/f1 increases."""
        if self.verbose:
            print(f"Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...")
        #model.save_pretrained("")
        #torch.save(args, os.path.join("", "training_args.bin"))
        torch.save(model.to('mps').state_dict(), 'besterror_rate_min_model')
        self.val_loss_min = val_loss

        # # Save model checkpoint (Overwrite)
        # if not os.path.exists(self.args.model_dir):
        #     os.makedirs(self.args.model_dir)
        # model_to_save = self.model.module if hasattr(self.model, 'module') else self.model
        # model_to_save.save_pretrained(self.args.model_dir)

        # # Save training arguments together with the trained model
        # torch.save(self.args, os.path.join(self.args.model_dir, 'training_args.bin'))
        # logger.info("Saving model checkpoint to %s", self.args.model_dir)

In [4]:
from prettytable import PrettyTable

def count_parameters2(model):
    table = PrettyTable(['Modules', 'Parameters'])
    total_params = 0
    for name, parameter in model.named_parameters():
        if not parameter.requires_grad: continue
        params = parameter.numel()
        table.add_row([name, params])
        total_params+=params
    #print(table)
    #print(f'Total Trainable Params: {total_params}')
    return total_params

# 3. 整理後データの読み込みと時点ずらし作業
財務データ：1994/12〜2021/11

配当込み収益率：1995/01〜2021/12

In [5]:
pickle_in = open("./inright_data.pickle","rb") #ファイルの読み込み, 配当込み収益率はランク正規化をしていないデータであることに注意
pickle_in_2 = open("./inleft_data.pickle","rb") #ファイルの読み込み
pickle_in_3 = open("./ranked_data.pickle","rb") #ファイルの読み込み
returns_data_4_0 = pickle.load(pickle_in)
returns_data_4 = returns_data_4_0.iloc[1:,0:].reset_index().iloc[:,1:] #1995/1~2021/12のデータを抽出
data_5_11_0 = pickle.load(pickle_in_2)
data_5_11 = data_5_11_0.iloc[:324,0:].reset_index().iloc[:,1:] #1994/12~2021/11のデータを抽出
data_5_7_0 = pickle.load(pickle_in_3)
data_5_7_1 = data_5_7_0.query('期日.str.contains("1994")', engine='python') #1994のデータを抽出
data_5_7_2 = data_5_7_1.index.values.tolist() #1994/12のデータのindexをlist型に変更
data_5_7 = data_5_7_0[~data_5_7_0.index.isin(data_5_7_2)] #1994/12のデータのindexにあうものを削除
pickle_in.close()
pickle_in_2.close()
pickle_in_3.close()
#display(returns_data_4), display(data_5_11), display(data_5_7)

In [6]:
day_num_5 = data_5_7.期日.nunique(dropna = True) #期日の数(月数)
com_num_5 = data_5_7.shape[0]/day_num_5 #分析可能な企業数
chara_num = data_5_11.shape[1]/int(com_num_5) #用いる企業特性の数
print('分析対象データの月数は',day_num_5,'、', '企業数は',com_num_5,'、', '企業特性数は',chara_num,'である。')

分析対象データの月数は 324 、 企業数は 1141.0 、 企業特性数は 40.0 である。


# 4. 入力データの準備

In [7]:
#右側ファクターネットワークの入力データ
x = returns_data_4.to_numpy()
#左側ベータネットワークの入力データ
y = data_5_11.iloc[:,0:].to_numpy()
x_2 = torch.tensor(x, dtype=torch.float32)
y_2 = torch.tensor(y, dtype=torch.float32)
print('右側ファクターネットワークの入力データの形状は',x.shape,'、', '左側ベータネットワークの入力データの形状は',y.shape,'である。')

右側ファクターネットワークの入力データの形状は (324, 1141) 、 左側ベータネットワークの入力データの形状は (324, 45640) である。


# 5. オートエンコーダークラスの生成

## 5.1 CA$_{1}$アーキテクチャ(左側に第1層を追加)

In [8]:
class Factors(nn.Module):
    def __init__(self):
        super(Factors, self).__init__()
        
        #層(layer：レイヤー)を定義
        
        ##右側(ファクターネットワーク)の層
        self.fc_r1 = nn.Linear( 
            N*1, #データ(特徴)の入力ユニット数(1059銘柄*1)
            K*1) #出力ユニット数(5ファクター*1)

        ##左側(ベータネットワーク)の層
        self.fc_l1 = nn.Linear(
            N*P, #データ(特徴)の入力ユニット数(1059銘柄*85企業特性)
            lhidden) #出力ユニット数(32)
        
        self.fc_l2 = nn.Linear(
            lhidden, #入力ユニット数(32)
            N*K) #出力結果への出力ユニット数(1059銘柄*5ファクター)
        
    def forward(self, x, y):
        # フォワードパスを定義
        
        #右側(ファクターネットワーク)のフォワードパスを定義
        x = (self.fc_r1(x))
        
        #左側(ベータネットワーク)のフォワードパスを定義
        y = F.relu(self.fc_l1(y)) #ReLU関数を適用
        y = F.relu(self.fc_l2(y)) #ReLU関数を適用
        
        #サイズ自動調整
        y = y.view(K, N) #サイズ自動調整
        x = x.view(1, K) #サイズ自動調整
        
        return x, y #返り値

#NNの初期値
N = int(com_num_5) #銘柄数(1059銘柄)
P = int(chara_num)   #企業特性数(85特性)
K = 5    #ファクター数
lhidden = 32  #左側隠れ層1での圧縮次元数

#モデル（Factorsクラス）のインスタンス化
model = Factors()

model = model.to(device)

print(model) #モデルの内容を出力

Factors(
  (fc_r1): Linear(in_features=1141, out_features=5, bias=True)
  (fc_l1): Linear(in_features=45640, out_features=32, bias=True)
  (fc_l2): Linear(in_features=32, out_features=5705, bias=True)
)


In [9]:
from prettytable import PrettyTable

def count_parameters2(model):
    table = PrettyTable(['Modules', 'Parameters'])
    total_params = 0
    for name, parameter in model.named_parameters():
        if not parameter.requires_grad: continue
        params = parameter.numel()
        table.add_row([name, params])
        total_params+=params
    #print(table)
    #print(f'Total Trainable Params: {total_params}')
    return total_params
count_parameters2(model)

1654487

## 5.2 CA$_{1}$アーキテクチャ(左側に第1層を追加、P×1圧縮あり)

In [10]:
class Factors(nn.Module):
    def __init__(self):
        super(Factors, self).__init__()
        
        #層(layer：レイヤー)を定義
        
        ##右側(ファクターネットワーク)の層
        self.fc_r1 = nn.Linear( 
            N*1, #データ(特徴)の入力ユニット数(1059銘柄*1)
            P*1) #出力ユニット数(93企業特性*1)
        
        self.fc_r2 = nn.Linear( 
            P*1, #データ(特徴)の入力ユニット数(85企業特性*1)
            K*1) #出力ユニット数(5ファクター*1)

        ##左側(ベータネットワーク)の層
        self.fc_l1 = nn.Linear(
            N*P, #データ(特徴)の入力ユニット数(1059銘柄*85企業特性)
            lhidden) #出力ユニット数(32)
        
        self.fc_l2 = nn.Linear(
            lhidden, #入力ユニット数(32)
            N*K) #出力結果への出力ユニット数(1059銘柄*5ファクター)
        
    def forward(self, x, y):
        # フォワードパスを定義
        
        #右側(ファクターネットワーク)のフォワードパスを定義
        x = (self.fc_r1(x))
        x = (self.fc_r2(x))
        
        #左側(ベータネットワーク)のフォワードパスを定義
        
        y = F.relu(self.fc_l1(y)) #ReLU関数を適用
        y = F.relu(self.fc_l2(y)) #ReLU関数を適用
        
        #サイズ自動調整
        
        y = y.view(K, N) #サイズ自動調整
        x = x.view(1, K) #サイズ自動調整
        
        return x, y #返り値

#NNの初期値
N = int(com_num_5) #銘柄数(1059銘柄)
P = int(chara_num)   #企業特性数(85特性)
K = 5    #ファクター数
lhidden = 32  #左側隠れ層1での圧縮次元数

#モデル（Factorsクラス）のインスタンス化
model = Factors()

model = model.to(device)

print(model) #モデルの内容を出力

Factors(
  (fc_r1): Linear(in_features=1141, out_features=40, bias=True)
  (fc_r2): Linear(in_features=40, out_features=5, bias=True)
  (fc_l1): Linear(in_features=45640, out_features=32, bias=True)
  (fc_l2): Linear(in_features=32, out_features=5705, bias=True)
)


# 6. 学習とバリデーション

## 6.1 学習とバリデーション関数の定義

In [11]:
num_workers = 1
num_month   = 1
x_batch = num_month
y_batch = num_month
criterion = nn.MSELoss()

#学習の関数
def train(model, optimizer, alpha, device):
    model.train()
    #for i_year in range(18):
    for i_year in range(2):
        xtrain_loader = torch.utils.data.DataLoader(x_2[:((i_year+1)*12),:], batch_size=x_batch, num_workers=num_workers, shuffle=False, pin_memory=True)
        ytrain_loader = torch.utils.data.DataLoader(y_2[:((i_year+1)*12),:], batch_size=y_batch, num_workers=num_workers, shuffle=False, pin_memory=True)
        for x_input, y_input in zip(xtrain_loader, ytrain_loader):
            x_input = x_input.to(device)
            y_input = y_input.to(device)
            u ,v = model(x_input.float(), y_input.float()) #モデルの出力を取得(左側出力v、右側出力uとする)
            uv = torch.mm(u, v) #最終的に積をとる
            loss = criterion(uv, x_input.float()) #入力x.float()と復元outputsの誤差を取得
            
            # パラメータのL1ノルムを損失関数に足す
            #l1 = sum(torch.norm(w , 1) for w in model.parameters()) #リスト内包表記
            l1 = sum(torch.norm(w/count_parameters2(model) , 1)  for w in model.parameters()) #リスト内包表記
            #loss = loss + alpha*l1.clone().detach()
            #loss = loss + alpha*l1.detach().clone()
            loss = loss + alpha*l1.detach()
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

    #print(x_input, y_input)
    #print('ここらで一区切りです')

#バリデーションの関数
def valid(model, alpha, device, xvalid_loader, yvalid_loader):
    model.eval() #model.train(mode=False)でも良い
    valid_loss = 0
    with torch.no_grad():
        for x_input, y_input in zip(xvalid_loader, yvalid_loader):
            x_input = x_input.to(device)
            y_input = y_input.to(device)
            u ,v = model(x_input.float(), y_input.float()) #モデルの出力を取得(左側出力v、右側出力uとする)
            uv = torch.mm(u, v) #最終的に積をとる
            loss = criterion(uv, x_input.float()) #入力x.float()と復元outputsの誤差を取得
            
            valid_loss += loss.item()  # 誤差(損失)の更新
            
        #1エポックあたりの損失を求める
        valid_loss = valid_loss/len(xvalid_loader)

    return valid_loss

## 6.2 最適化関数の定義

In [12]:
#最適化関数(specify loss function)をAdamに設定
def get_optimizer(trial, model):
    adam_lr = trial.suggest_loguniform('adam_lr', 1e-10, 1e-1) #学習率を変化させる
    #weight_decay = trial.suggest_loguniform('weight_decay', 1e-10, 1e-3)
    optimizer = optim.Adam(model.parameters(), lr=adam_lr, weight_decay=0) #weight_decayは0
    return optimizer

## 6.3 目的関数の定義

In [13]:
# エポック数
n_epochs = 5
error_rate_min = 0.0
def objective(trial):
    global i_trial
    print('今は', i_trial ,'回目のトライアル')
    optimizer = get_optimizer(trial, model)
    alpha = trial.suggest_loguniform('alpha', 1e-10, 1e-1)
    early_stopping = EarlyStopping(patience=5, verbose=True) #ここでインスタンス化
    #学習の実行
    for epoch in range(1, n_epochs+1):
        print('Epoch: {}'.format(epoch))
        train_loss = 0.0
        train(model, optimizer, alpha, device)
        error_rate = valid(model, alpha, device, xvalid_loader, yvalid_loader)
        
        early_stopping(error_rate, model) # 最良モデルならモデルパラメータ保存
        if early_stopping.early_stop: 
            # 一定epochだけval_lossが最低値を更新しなかった場合、ここに入り学習を終了
            break
    
    global error_rate_min
    if i_trial == 0:
        #error_rate_min = error_rate
        error_rate_min = early_stopping.val_loss_min
        model.load_state_dict(torch.load('besterror_rate_min_model'))
        torch.save(model.to('mps').state_dict(), 'error_rate_min_model')
        #print(list(model.parameters())[0]) #確認用
    else:
        if error_rate < error_rate_min:
            #error_rate_min = error_rate
            error_rate_min = early_stopping.val_loss_min
            model.load_state_dict(torch.load('besterror_rate_min_model'))
            torch.save(model.to('mps').state_dict(), 'error_rate_min_model')
            print('model updated')
            #print(list(model.parameters())[0]) #確認用
    i_trial += 1
    return error_rate

## 6.4 学習とバリデーション(パラメータチューニング)の実行
ハイパーパラメーターは学習率とl1の2つ

In [14]:
TRIAL_SIZE = 3
i_year_2 = 4 #バリデーション期間の年数
xvalid_loader = torch.utils.data.DataLoader(x_2[216:216+(i_year_2*12),:], batch_size=x_batch, num_workers=num_workers, shuffle=False ,pin_memory=True)
yvalid_loader = torch.utils.data.DataLoader(y_2[216:216+(i_year_2*12),:], batch_size=y_batch, num_workers=num_workers, shuffle=False ,pin_memory=True)

i_trial = 0
study = optuna.create_study(sampler=optuna.samplers.TPESampler(seed=0)) #123
study.optimize(objective, n_trials=TRIAL_SIZE)
#最適化したハイパーパラメータの結果
best_params = study.best_params
f = open('best_params.txt', 'w')
f.write(str(best_params))
f.close()

[I 2022-12-15 18:01:39,815] A new study created in memory with name: no-name-c5faf2d6-c051-4622-868a-4ab11c0f9b22


今は 0 回目のトライアル
Epoch: 1
Validation loss decreased (inf --> 0.009402).  Saving model ...
Epoch: 2
Validation loss decreased (0.009402 --> 0.009170).  Saving model ...
Epoch: 3
Validation loss decreased (0.009170 --> 0.008951).  Saving model ...
Epoch: 4
Validation loss decreased (0.008951 --> 0.008730).  Saving model ...
Epoch: 5


[I 2022-12-15 18:01:55,909] Trial 0 finished with value: 0.00854461468406953 and parameters: {'adam_lr': 8.696040132105582e-06, 'alpha': 0.00027334069690310554}. Best is trial 0 with value: 0.00854461468406953.


Validation loss decreased (0.008730 --> 0.008545).  Saving model ...
今は 1 回目のトライアル
Epoch: 1
Validation loss decreased (inf --> 0.008107).  Saving model ...
Epoch: 2
Validation loss decreased (0.008107 --> 0.007840).  Saving model ...
Epoch: 3
Validation loss decreased (0.007840 --> 0.007600).  Saving model ...
Epoch: 4
Validation loss decreased (0.007600 --> 0.007472).  Saving model ...
Epoch: 5


[I 2022-12-15 18:02:11,381] Trial 1 finished with value: 0.0074058735626749694 and parameters: {'adam_lr': 2.6599310838681845e-05, 'alpha': 8.015832747965139e-06}. Best is trial 1 with value: 0.0074058735626749694.


Validation loss decreased (0.007472 --> 0.007406).  Saving model ...
model updated
今は 2 回目のトライアル
Epoch: 1
Validation loss decreased (inf --> 0.007404).  Saving model ...
Epoch: 2
Validation loss decreased (0.007404 --> 0.007402).  Saving model ...
Epoch: 3
Validation loss decreased (0.007402 --> 0.007400).  Saving model ...
Epoch: 4
Validation loss decreased (0.007400 --> 0.007398).  Saving model ...
Epoch: 5


[I 2022-12-15 18:02:26,893] Trial 2 finished with value: 0.007396543747745454 and parameters: {'adam_lr': 6.499698237449664e-07, 'alpha': 6.50200078509766e-05}. Best is trial 2 with value: 0.007396543747745454.


Validation loss decreased (0.007398 --> 0.007397).  Saving model ...
model updated
